In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy
import scipy.optimize
import scipy.stats

In [ ]:
x_values = numpy.array([0, 15, 30, 45, 60])
y_values = numpy.array([70, 77, 83, 90, 100])
y_errors = numpy.array([0.8, 0.7, 1.2, 1.1, 1.2])

#x_values, y_values, y_errors = numpy.loadtxt(filename, unpack=True) # read columns of data from a file

In [ ]:
assert len(y_values) == len(x_values)
assert len(y_errors) == len(y_values)

plt.figure(figsize=(5,3))
plt.errorbar(x_values, 
             y_values, 
             yerr=y_errors, # use y_errors array for y error bars
             marker='o',    # circular markers at each datapoint
             linestyle='None') # no connecting lines

plt.xlabel('x data (units)') # axis labels and units
plt.ylabel('y data (units)')
plt.show() 

In [ ]:
def model_function(x, *params):
    return params[0]*x + params[1]

In [ ]:
initial_values = numpy.array([0.5, 70.0]) # Initial guess for fit parameters

In [ ]:
popt, cov = scipy.optimize.curve_fit(model_function, # function to fit
                                     x_values, # x data
                                     y_values, # y data
                                     sigma=y_errors, # array of error bars for the fit
                                     absolute_sigma=True, # errors bars DO represent 1 std error
                                     p0=initial_values, # starting point for fit
                                     check_finite=True) # raise ValueError if NaN encountered (don't allow errors to pass)


plt.figure(figsize=(5,3))
plt.errorbar(x_values, 
             y_values, 
             yerr=y_errors, 
             marker='o', 
             linestyle='None')
plt.xlabel('x data (units)') # Axis labels
plt.ylabel('y data (units)')

smooth_x = numpy.linspace(x_values[0], x_values[-1], 1000) # more points, over range of data
plt.plot(smooth_x, 
         model_function(smooth_x , *popt), 
         color='r')
plt.show()

In [ ]:
def chi_squared(model_params, model, x_data, y_data, y_err):
    return numpy.sum(((y_data - model(x_data, *model_params))/y_err)**2) # Note the `*model_params' here!


chi_squared_min = chi_squared(popt, model_function, x_values, y_values, y_errors)
print('chi^2_min = {}'.format(chi_squared_min))

degrees_of_freedom = x_values.size - popt.size
print('reduced chi^2 = {}'.format(chi_squared_min/degrees_of_freedom))
print('P(chi^2_min, DoF) = {}'.format(scipy.stats.chi2.sf(chi_squared_min, degrees_of_freedom)))

popt_errs = numpy.sqrt(numpy.diag(cov))

for i, (val, err) in enumerate(zip(popt, popt_errs)):
    print('optimised parameter[{}] = ({} +/- {}) units'.format(i, val, err))
    